<a href="https://colab.research.google.com/github/ontenkutsenko/NLP_Airbnb_Delisting_Prediction/blob/main/Translation%20attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [3]:
!gdown 1Y63uPJCSWvlefU-HYpuVumaiqgjHjpb8
!unzip '/content/Project Corpora.zip'
!rm '/content/Project Corpora.zip'

Downloading...
From (original): https://drive.google.com/uc?id=1Y63uPJCSWvlefU-HYpuVumaiqgjHjpb8
From (redirected): https://drive.google.com/uc?id=1Y63uPJCSWvlefU-HYpuVumaiqgjHjpb8&confirm=t&uuid=2cb1302a-2134-4bad-9993-3fd55afb372c
To: /content/Project Corpora.zip
100% 55.4M/55.4M [00:00<00:00, 73.0MB/s]
Archive:  /content/Project Corpora.zip
  inflating: Project Corpora/test.xlsx  
  inflating: Project Corpora/test_reviews.xlsx  
  inflating: Project Corpora/train.xlsx  
  inflating: Project Corpora/train_reviews.xlsx  


In [4]:
df_train = pd.read_excel('/content/Project Corpora/train.xlsx')
df_train_reviews = pd.read_excel('/content/Project Corpora/train_reviews.xlsx')
df_test = pd.read_excel('/content/Project Corpora/test.xlsx')
df_test_reviews = pd.read_excel('/content/Project Corpora/test_reviews.xlsx')

In [5]:
from transformers import pipeline, XLMRobertaForSequenceClassification, XLMRobertaTokenizer

# Load the pre-trained model checkpoint and tokenizer
model = XLMRobertaForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")
tokenizer = XLMRobertaTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")


pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)  # Specify GPU device if available




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

In [6]:
text = df_train_reviews.comments.to_list()[:10000]
labels = []

for i in tqdm(text):
  labels.append(pipe(i, top_k=1, truncation=True)[0]['label'])

100%|██████████| 10000/10000 [02:00<00:00, 83.20it/s]


In [7]:
df_experiment = df_train_reviews[:10000]

In [8]:
df_experiment['language'] = labels

<ipython-input-8-5f11783fc29e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_experiment['language'] = labels


In [9]:
df_experiment = df_experiment.sort_values(by = 'language')

In [10]:
df_experiment.language.value_counts()

language
en    6353
fr    1482
pt     728
es     596
de     380
it     160
nl     101
ru      64
ja      42
zh      30
ur      22
pl      14
hi      12
tr       9
sw       4
bg       1
el       1
ar       1
Name: count, dtype: int64

In [12]:
df_ex = df_experiment.copy()

In [13]:
df_experiment = df_experiment[df_experiment['language']=='de']

In [14]:
df_experiment

,index,comments,language
8132,171,Der ideale Ort zum Rekaxen,de
9603,204,Wir sind zum ersten Mal as Familie mit zwei gr...,de
9845,207,"Die Wohung von Clement liegt perfekt, nur weni...",de
1569,30,Der Garten und der Pool sind ein Traum. Den Po...,de
5711,119,Gerne wieder beim nächsten Aufenthalt in Lissabon,de
...,...,...,...
9071,194,Tolle Lage und schöne Wohnung. Viel schöne Res...,de
7580,161,Wir haben uns im Castro Beach House wirklich s...,de
6883,146,Die Kommunikation war super alles hat schnell ...,de
5140,100,angenehm und unkompliziert. die wohnung ist ei...,de


In [31]:
df_experiment.iloc[0]['language']

'de'

In [16]:
from tqdm import tqdm

translations = []

lang = df_experiment.iloc[0]['language']

for i in tqdm(range(len(df_experiment))):
  if df_experiment['language'].iloc[i] == 'en':
    translations.append(df_experiment['comments'].iloc[i])
  elif df_experiment['language'].iloc[i] != lang:
    lang = df_experiment['language'].iloc[i]
    print('\n New language : ', lang, '\n')
    try:
      tokenizer = AutoTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang}-en")
      model = AutoModelForSeq2SeqLM.from_pretrained(f"Helsinki-NLP/opus-mt-{lang}-en")
      pipe = pipeline("translation", model=model, tokenizer=tokenizer, device = 0)
      translations.append(pipe(df_experiment['comments'].iloc[i], max_length=10000)[0]['translation_text'])
    except:
      translations.append('')
  else:
      translations.append(pipe(df_experiment['comments'].iloc[i], max_length=10000)[0]['translation_text'])
      translations.append('')

  0%|          | 0/380 [00:00<?, ?it/s]


KeyError: 'translation_text'

In [18]:
pipe(df_experiment['comments'].iloc[i], max_length=10000)

[{'label': 'de', 'score': 0.9907051920890808}]

In [33]:
translations

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [16]:
df_experiment['tr'] = translations

In [22]:
df_experiment[df_experiment['language'] == 'de']

,index,comments,language,tr
8132,171,Der ideale Ort zum Rekaxen,de,
9603,204,Wir sind zum ersten Mal as Familie mit zwei gr...,de,
9845,207,"Die Wohung von Clement liegt perfekt, nur weni...",de,
1569,30,Der Garten und der Pool sind ein Traum. Den Po...,de,
5711,119,Gerne wieder beim nächsten Aufenthalt in Lissabon,de,
...,...,...,...,...
9071,194,Tolle Lage und schöne Wohnung. Viel schöne Res...,de,
7580,161,Wir haben uns im Castro Beach House wirklich s...,de,
6883,146,Die Kommunikation war super alles hat schnell ...,de,
5140,100,angenehm und unkompliziert. die wohnung ist ei...,de,


In [19]:
len(translations)

10000

In [20]:
df_experiment.tr.value_counts()

tr
                                                                                                                                                                                                                                                                                                                                                                                                                                                                              3647
.                                                                                                                                                                                                                                                                                                                                                                                                                                                                               15
Great place!                                                   

In [4]:
translator = pipeline("translation", model=model, tokenizer=tokenizer)

# Example sentence to translate
sentence = "Привет, как дела?"

# Translate the sentence
translated_sentence = translator(sentence, max_length=50)[0]['translation_text']

# Print the translated sentence
print("Translated Sentence:", translated_sentence)


Translated Sentence: Hey, how's it going?


In [27]:
a

'Si'

In [5]:
sentences_russian = [
    "Я люблю гулять по парку вечером.",
    "Сегодня очень хорошая погода.",
    "У меня много работы на этой неделе.",
    "Мой друг приезжает в гости завтра.",
    "Я хочу посмотреть новый фильм в кино.",
    "Моя сестра учится в университете.",
    "Вчера я готовил пасту с морепродуктами.",
    "Он хочет научиться играть на гитаре.",
    "Утром всегда пью кофе с молоком.",
    "Я люблю читать книги на свежем воздухе.",
    "В субботу планирую посетить музей.",
    "Мы с друзьями собираемся на пикник на выходных.",
    "Моя собака очень ласковая и дружелюбная.",
    "Она хочет поехать в отпуск на море.",
    "Я мечтаю научиться говорить на английском свободно.",
    "Сегодня вечером я иду в театр с мамой.",
    "У нас в саду растут красивые цветы.",
    "Летом мы часто ездим на дачу.",
    "Вчера я посетил новую кафе в центре города.",
    "Он хочет купить новый автомобиль этой осенью."
]

for i in sentences_russian:
  translated_sentence = translator(i, max_length=50)[0]['translation_text']

  # Print the translated sentence
  print("Translated Sentence:", translated_sentence)


Translated Sentence: I like to walk in the park tonight.
Translated Sentence: It's very good weather today.
Translated Sentence: I have a lot of work to do this week.
Translated Sentence: My friend's coming over tomorrow.
Translated Sentence: I want to see a new movie in the movies.
Translated Sentence: My sister goes to university.
Translated Sentence: I cooked pasta with seafood yesterday.
Translated Sentence: He wants to learn to play the guitar.
Translated Sentence: I always drink coffee and milk in the morning.
Translated Sentence: I like reading books in the fresh air.
Translated Sentence: I'm planning to visit the museum on Saturday.
Translated Sentence: My friends and I are going on a picnic this weekend.
Translated Sentence: My dog is very sweet and friendly.
Translated Sentence: She wants to go on vacation at sea.
Translated Sentence: I dream of learning to speak English freely.
Translated Sentence: I'm going to the theater with my mom tonight.
Translated Sentence: Beautiful 